In [ ]:
import wandb

In [ ]:
import pickle

In [ ]:
import argparse, os, logging, random, time
import numpy as np
import math
import time
import scipy.sparse
import lightgbm as lgb
import data_helpers as dh

import sklearn.metrics

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
# import tensorboardX as tbx
from sklearn.utils.extmath import softmax

from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import Optimizer, AdamW, SGD
import gc


import pdb

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    type_prefix = torch.cuda
else:
    type_prefix = torch

In [ ]:
def one_hot(y, numslot, mask=None):
    y_tensor = y.type(type_prefix.LongTensor).reshape(-1, 1)
    y_one_hot = torch.zeros(y_tensor.size()[0], numslot, device=device, dtype=torch.float32, requires_grad=False).scatter_(1, y_tensor, 1)
    if mask is not None:
        y_one_hot = y_one_hot * mask
    y_one_hot = y_one_hot.reshape(y.shape[0], -1)
    return y_one_hot

In [ ]:
class BatchDense(nn.Module):
    def __init__(self, batch, in_features, out_features, bias_init=None):
        super(BatchDense, self).__init__()
        self.batch = batch
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(batch, in_features, out_features))
        self.bias = Parameter(torch.Tensor(batch, 1, out_features))
        self.reset_parameters(bias_init)
    def reset_parameters(self, bias_init=None):
        stdv = math.sqrt(6.0 /(self.in_features + self.out_features))
        self.weight.data.uniform_(-stdv, stdv)
        if bias_init is not None:
            self.bias.data = torch.from_numpy(bias_init)
        else:
            self.bias.data.fill_(0)
    def forward(self, x):
        size = x.size()
        # Todo: avoid the swap axis
        x = x.view(x.size(0), self.batch, -1)
        out = x.transpose(0, 1).contiguous()
        out = torch.baddbmm(self.bias, out, self.weight)
        out = out.transpose(0, 1).contiguous()
        out = out.view(x.size(0), -1)
        return out

In [ ]:
class GBDT2NN(nn.Module):
    def __init__(self, input_size, used_features,
                 tree_layers, output_w, output_b, task):
        super(GBDT2NN, self).__init__()
        print('Init GBDT2NN')
        self.task = task
        self.n_models = len(used_features)
        self.tree_layers = tree_layers
        n_feature = len(used_features[0])
        used_features = np.asarray(used_features).reshape(-1)
        self.used_features = Variable(torch.from_numpy(used_features).to(device), requires_grad=False)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        assert len(tree_layers) > 0
        self.bdenses = nn.ModuleList()
        self.bns = nn.ModuleList()
        self.bdenses.append(BatchDense(self.n_models, n_feature, tree_layers[0]))
        for i in range(1, len(tree_layers)):
            self.bdenses.append(BatchDense(self.n_models, tree_layers[i-1], tree_layers[i]))
        for i in range(len(tree_layers)-1):
            self.bns.append(nn.BatchNorm1d(tree_layers[i] * self.n_models))
        self.out_weight = Variable(torch.from_numpy(output_w).to(device), requires_grad=False)
        self.out_bias = Variable(torch.from_numpy(output_b).to(device), requires_grad=False)
        print('Init GBDT2NN succeed!')
        if self.task == 'regression':
            self.criterion = nn.MSELoss()
        else:
            self.criterion = nn.BCELoss()

    def batchmul(self, x, f):
        out = x.view(x.size(0), self.n_models, -1)
        out = f(out)
        out = out.view(x.size(0), -1)
        return out
    def lastlayer(self, x):
        out = torch.index_select(x, dim=1, index=self.used_features)
        for i in range(len(self.bdenses) - 1):
            out = self.batchmul(out, self.bdenses[i])
            out = self.bns[i](out)
            out = self.relu(out)
        return out
    
    def forward(self, x):
        out = self.lastlayer(x.float())
        pred = self.batchmul(out, self.bdenses[-1])
        out = torch.addmm(self.out_bias, pred, self.out_weight)
        if self.task != 'regression':
            return self.sigmoid(out), pred
        return out, pred

    def emb_loss(self, emb_pred, emb_target):
        loss_weight = torch.abs(torch.sum(self.out_weight, 1))
        l2_loss = nn.MSELoss(reduction='none')(emb_pred, emb_target)*loss_weight
        return torch.mean(torch.sum(l2_loss, dim=1))

    def joint_loss(self, out, target, emb_pred, emb_target, ratio):
        return (1-ratio) * self.criterion(out, target) + ratio * self.emb_loss(emb_pred, emb_target)

    def true_loss(self, out, target):
        return self.criterion(out, target)


In [ ]:
def eval_metrics(task, true, pred):
    if task == 'binary':
        logloss = sklearn.metrics.log_loss(true.astype(np.float64), pred.astype(np.float64))
        auc = sklearn.metrics.roc_auc_score(true, pred)
        # error = 1-sklearn.metrics.accuracy_score(true,(pred+0.5).astype(np.int32))
        return (logloss, auc)#, error)
    else:
        mseloss = sklearn.metrics.mean_squared_error(true, pred)
        return mseloss

def EvalTestset(test_x, test_y, model, test_batch_size, test_x_opt=None):
    test_len = test_x.shape[0]
    test_num_batch = math.ceil(test_len / test_batch_size)
    sum_loss = 0.0
    y_preds = []
    model.eval()
    with torch.no_grad():
        for jdx in range(test_num_batch):
            tst_st = jdx * test_batch_size
            tst_ed = min(test_len, tst_st + test_batch_size)
            inputs = torch.from_numpy(test_x[tst_st:tst_ed].astype(np.float32)).to(device)
            if test_x_opt is not None:
                inputs_opt = torch.from_numpy(test_x_opt[tst_st:tst_ed].astype(np.float32)).to(device)
                outputs = model(inputs, inputs_opt)
            else:
                outputs = model(inputs)
            targets = torch.from_numpy(test_y[tst_st:tst_ed]).to(device)
            if isinstance(outputs, tuple):
                outputs = outputs[0]
            y_preds.append(outputs)
            loss_tst = model.true_loss(outputs, targets).item()
            sum_loss += (tst_ed - tst_st) * loss_tst
    return sum_loss / test_len, np.concatenate(y_preds, 0)

def TrainWithLog(loss_dr, loss_init, loss_de, log_freq, test_freq, task, test_batch_size,                
                train_x, train_y, 
                 train_y_opt, test_x, test_y, model, opt,
                 epoch, batch_size, n_output, key="",
                 train_x_opt=None, test_x_opt=None):
    # trn_writer = tf.summary.FileWriter(summaryPath+plot_title+key+"_output/train")
    # tst_writer = tf.summary.FileWriter(summaryPath+plot_title+key+"_output/test")
    if isinstance(test_x, scipy.sparse.csr_matrix):
        test_x = test_x.todense()
    train_len = train_x.shape[0]
    global_iter = 0
    trn_batch_size = batch_size
    train_num_batch = math.ceil(train_len / trn_batch_size)
    total_iterations = epoch * train_num_batch
    start_time = time.time()
    total_time = 0.0
    min_loss = float("Inf")
    # min_error = float("Inf")
    max_auc = 0.0
    for epoch in range(epoch):
        shuffled_indices = np.random.permutation(np.arange(train_x.shape[0]))
        Loss_trn_epoch = 0.0
        Loss_trn_log = 0.0
        log_st = 0
        for local_iter in range(train_num_batch):
            trn_st = local_iter * trn_batch_size
            trn_ed = min(train_len, trn_st + trn_batch_size)
            batch_trn_x = train_x[shuffled_indices[trn_st:trn_ed]]
            if isinstance(batch_trn_x, scipy.sparse.csr_matrix):
                batch_trn_x = batch_trn_x.todense()
            inputs = torch.from_numpy(batch_trn_x.astype(np.float32)).to(device)
            targets = torch.from_numpy(train_y[shuffled_indices[trn_st:trn_ed],:]).to(device)
            model.train()
            if train_x_opt is not None:
                inputs_opt = torch.from_numpy(train_x_opt[shuffled_indices[trn_st:trn_ed]].astype(np.float32)).to(device)
                outputs = model(inputs, inputs_opt)
            else:
                outputs = model(inputs)
            opt.zero_grad()
            if isinstance(outputs, tuple) and train_y_opt is not None:
                # targets_inner = torch.from_numpy(s_train_y_opt[trn_st:trn_ed,:]).to(device)
                targets_inner = torch.from_numpy(train_y_opt[shuffled_indices[trn_st:trn_ed],:]).to(device)
                loss_ratio = loss_init * max(0.3,loss_dr ** (epoch // loss_de))#max(0.5, args.loss_dr ** (epoch // args.loss_de))
                if len(outputs) == 3:
                    loss_val = model.joint_loss(outputs[0], targets, outputs[1], targets_inner, loss_ratio, outputs[2])
                else:
                    loss_val = model.joint_loss(outputs[0], targets, outputs[1], targets_inner, loss_ratio)
                loss_val.backward()
                loss_val = model.true_loss(outputs[0], targets)
            elif isinstance(outputs, tuple):
                loss_val = model.true_loss(outputs[0], targets)
                loss_val.backward()
            else:
                loss_val = model.true_loss(outputs, targets)
                loss_val.backward()
            opt.step()
            loss_val = loss_val.item()
            wandb.log({"batch loss":loss_val})
            global_iter += 1
            Loss_trn_epoch += (trn_ed - trn_st) * loss_val
            Loss_trn_log += (trn_ed - trn_st) * loss_val
            if global_iter % log_freq == 0:
                print(key+"Epoch-{:0>3d} {:>5d} Batches, Step {:>6d}, Training Loss: {:>9.6f} (AllAvg {:>9.6f})"
                            .format(epoch, local_iter + 1, global_iter, Loss_trn_log/(trn_ed-log_st), Loss_trn_epoch/trn_ed))
                
                # trn_summ = tf.Summary()
                # trn_summ.value.add(tag=args.data+ "/Train/Loss", simple_value = Loss_trn_log/(trn_ed-log_st))
                # trn_writer.add_summary(trn_summ, global_iter)
                log_st = trn_ed
                Loss_trn_log = 0.0
            if global_iter % test_freq == 0 or local_iter == train_num_batch - 1:
                if model == 'deepgbm' or model == 'd1':
                    try:
                        print('Alpha: '+str(model.alpha))
                        print('Beta: '+str(model.beta))
                    except:
                        pass
                # tst_summ = tf.Summary()
                torch.cuda.empty_cache()
                test_loss, pred_y = EvalTestset(test_x, test_y, model, test_batch_size, test_x_opt)
                wandb.log({"loss":test_loss})
                current_used_time = time.time() - start_time
                start_time = time.time()
                wandb.log({"createdAt":start_time})
                total_time += current_used_time
                remaining_time = (total_iterations - (global_iter) ) * (total_time / (global_iter))
                if task == 'binary':
                    metrics = eval_metrics(task, test_y, pred_y)
                    _, test_auc = metrics
                    wandb.log({"test batch auc":test_auc})
                    # min_error = min(min_error, test_error)
                    max_auc = max(max_auc, test_auc)
                    wandb.log({"test max auc":max_auc})
                    # tst_summ.value.add(tag=args.data+"/Test/Eval/Error", simple_value = test_error)
                    # tst_summ.value.add(tag=args.data+"/Test/Eval/AUC", simple_value = test_auc)
                    # tst_summ.value.add(tag=args.data+"/Test/Eval/Min_Error", simple_value = min_error)
                    # tst_summ.value.add(tag=args.data+"/Test/Eval/Max_AUC", simple_value = max_auc)
                    print(key+"Evaluate Result:\nEpoch-{:0>3d} {:>5d} Batches, Step {:>6d}, Testing Loss: {:>9.6f}, Testing AUC: {:8.6f}, Used Time: {:>5.1f}m, Remaining Time: {:5.1f}m"
                            .format(epoch, local_iter + 1, global_iter, test_loss, test_auc, total_time/60.0, remaining_time/60.0))
                else:
                    print(key+"Evaluate Result:\nEpoch-{:0>3d} {:>5d} Batches, Step {:>6d}, Testing Loss: {:>9.6f}, Used Time: {:>5.1f}m, Remaining Time: {:5.1f}m"
                            .format(epoch, local_iter + 1, global_iter, test_loss, total_time/60.0, remaining_time/60.0))
                min_loss = min(min_loss, test_loss)
                wandb.log({"test min loss": min_loss})
                # tst_summ.value.add(tag=args.data+"/Test/Loss", simple_value = test_loss)
                # tst_summ.value.add(tag=args.data+"/Test/Min_Loss", simple_value = min_loss)
                print("-------------------------------------------------------------------------------")
                # tst_writer.add_summary(tst_summ, global_iter)
                # tst_writer.flush()
        print("Best Metric: %s"%(str(max_auc) if task=='binary' else str(min_loss)))
        print("####################################################################################")
    print("Final Best Metric: %s"%(str(max_auc) if task=='binary' else str(min_loss)))
    return min_loss        

def GetEmbPred(model, fun, X, test_batch_size):
    model.eval()
    tst_len = X.shape[0]
    test_num_batch = math.ceil(tst_len / test_batch_size)
    y_preds = []
    with torch.no_grad():
        for jdx in range(test_num_batch):
            tst_st = jdx * test_batch_size
            tst_ed = min(tst_len, tst_st + test_batch_size)
            inputs = torch.from_numpy(X[tst_st:tst_ed]).to(device)
            t_preds = fun(inputs).data.cpu().numpy()
            y_preds.append(t_preds)
        y_preds = np.concatenate(y_preds, 0)
    return y_preds


In [ ]:
HOME_DIR = os.getcwd()
DATA_DIR = os.path.join(HOME_DIR, 'data')

In [ ]:
num_data = dh.load_data('/work/neurotrees/articles code reproduction/DeepGBM/data/data_offline_num')

2022-02-07 10:00:26,897 [INFO] data loaded.
 train_x shape: (3918, 12). train_y shape: (3918, 1).
 test_x shape: (980, 12). test_y shape: (980, 1).


In [ ]:
train_x, train_y, test_x, test_y = num_data

In [ ]:
train_x.shape

(3918, 12)

In [ ]:
type(train_x)

numpy.ndarray

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'max_epoch': {
            'values': [20, 50, 100, 150]
        },
        'batch_size': {
            'values': [512, 256, 128, 64]
        },
        
        'lr': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        'l2_reg': {
            'values': [1e-2, 1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
        
        'optimizer': {
            'values': ['adamW', 'sgd']
        },
    }
}

In [ ]:

# sweep1
# {lr = 1e-3,
# l2_reg = 1e-6,
# max_epoch = 100,
# batch_size = 512,
# test_batch_size = 100,} 

# sweep2
# tree_layers = [100,100,100,50],
# embsize = 20,
    

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="deepgbm-wandb")

2022-02-07 10:00:27,642 [INFO] Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Create sweep with ID: 8s53m07f
Sweep URL: https://wandb.ai/iloncka/deepgbm-wandb/sweeps/8s53m07f


In [ ]:
PATH_TO_PICKLE = '/work/neurotrees/experiments/DeepGBM-decomposition/wine-dataset'

In [ ]:
def train():
    with open(os.path.join(PATH_TO_PICKLE,'n_models_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        n_models = pickle.load(f)
    
    with open(os.path.join(PATH_TO_PICKLE,'max_ntree_per_split_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        max_ntree_per_split = pickle.load(f)
        
    with open(os.path.join(PATH_TO_PICKLE,'group_average_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        group_average = pickle.load(f)

    with open(os.path.join(PATH_TO_PICKLE,'leaf_preds_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        leaf_preds = pickle.load(f)
        
    with open(os.path.join(PATH_TO_PICKLE,'test_leaf_preds_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        test_leaf_preds = pickle.load(f)
        
    with open(os.path.join(PATH_TO_PICKLE,'tree_outputs_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        tree_outputs = pickle.load(f) 

    with open(os.path.join(PATH_TO_PICKLE,'used_features_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        used_features = pickle.load(f)

    with open(os.path.join(PATH_TO_PICKLE,'train_embs_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        train_embs = pickle.load(f)
    
    with open(os.path.join(PATH_TO_PICKLE,'output_w_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        output_w = pickle.load(f)
        
    with open(os.path.join(PATH_TO_PICKLE,'output_b_wine_100.pickle'), 'rb') as f:
        # Pickle using the highest protocol available.
        output_b = pickle.load(f)


    config_defaults = dict(      
    loss_init = 1.0,
    loss_dr = 0.7,
    loss_de = 2,
    log_freq = 500,
    test_freq = 300,
    key = "",
    seeds = [1], #2,3,4,5 
    tree_layers = [100,100,100,50],
    lr = 1e-3,
    l2_reg = 1e-6,
    max_epoch = 100,
    batch_size = 512,
    test_batch_size = 100, 
    embsize = 20,
    task = "regression",
    n_output = train_y.shape[1]
    )


    # Initialize a new wandb run
    wandb.init(config=config_defaults)
        
    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
 
    # tree_layers = [int(x) for x in tree_layers.split(',')]
    config.tree_layers.append(config.embsize)

    concate_train_x = np.concatenate([train_x, np.zeros((train_x.shape[0],1), dtype=np.float32)], axis=-1)
    concate_test_x = np.concatenate([test_x, np.zeros((test_x.shape[0],1), dtype=np.float32)], axis=-1)
    tree_outputs = train_embs
    for seed in config.seeds:
        np.random.seed(seed)
        torch.cuda.manual_seed_all(seed)
        gbdt2nn_model = GBDT2NN(concate_train_x.shape[1], 
                                np.asarray(used_features,dtype=np.int64),
                                config.tree_layers,
                                output_w, output_b, config.task).to(device)
        if config.optimizer=='sgd':
            opt = SGD(gbdt2nn_model.parameters(),lr=config.lr, momentum=0.9)
        elif config.optimizer=='adamW':
            opt = AdamW(gbdt2nn_model.parameters(),lr=config.lr, weight_decay=config.l2_reg, amsgrad=False)
       

        TrainWithLog(config.loss_dr, config.loss_init, config.loss_de, config.log_freq, 
                    config.test_freq, config.task, config.test_batch_size,
                    concate_train_x, train_y, tree_outputs,
                    concate_test_x, test_y, gbdt2nn_model, opt,
                    config.max_epoch, config.batch_size, config.n_output, config.key)
        _,pred_y = EvalTestset(concate_test_x, test_y, gbdt2nn_model, config.test_batch_size)
        metric = eval_metrics(config.task, test_y, pred_y)
        print('Final metrics: %s'%str(metric))

In [ ]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: xf9y0v77 with config:
wandb: 	batch_size: 512
wandb: 	l2_reg: 3e-05
wandb: 	lr: 0.001
wandb: 	max_epoch: 20
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Init GBDT2NN
Init GBDT2NN succeed!
Evaluate Result:
Epoch-000     8 Batches, Step      8, Testing Loss: 29.686865, Used Time:   0.0m, Remaining Time:   0.1m
-------------------------------------------------------------------------------
Best Metric: 29.686864697203344
####################################################################################
Evaluate Result:
Epoch-001     8 Batches, Step     16, Testing Loss: 29.956185, Used Time:   0.0m, Remaining Time:   0.1m
-------------------------------------------------------------------------------
Best Metric: 29.686864697203344
####################################################################################
Evaluate Result:
Epoch-002     8 Batches, Step     24, Testing Loss:  5.428142, Used Time:   0.0m, Remaining Time:   0.1m
-------------------------------------------------------------------------------
Best Metric: 5.428142080501634
####################################################################################
Evaluate 

batch loss,▇▇▇██▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
createdAt,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,██▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test min loss,██▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch loss,0.12653
createdAt,1644228484.19434
loss,0.26512
test min loss,0.26492


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: vv5wxr2p with config:
wandb: 	batch_size: 256
wandb: 	l2_reg: 0.0001
wandb: 	lr: 0.0001
wandb: 	max_epoch: 150
wandb: 	optimizer: adamW
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Init GBDT2NN
Init GBDT2NN succeed!
Evaluate Result:
Epoch-000    16 Batches, Step     16, Testing Loss: 35.096963, Used Time:   0.0m, Remaining Time:   1.1m
-------------------------------------------------------------------------------
Best Metric: 35.09696306501116
####################################################################################
Evaluate Result:
Epoch-001    16 Batches, Step     32, Testing Loss: 31.897870, Used Time:   0.0m, Remaining Time:   1.1m
-------------------------------------------------------------------------------
Best Metric: 31.897869849691585
####################################################################################
Evaluate Result:
Epoch-002    16 Batches, Step     48, Testing Loss: 24.634640, Used Time:   0.0m, Remaining Time:   1.1m
-------------------------------------------------------------------------------
Best Metric: 24.634639545362823
####################################################################################
Evaluate 

...

wandb: Agent Starting Run: ze0hq9dv with config:
wandb: 	batch_size: 512
wandb: 	l2_reg: 0.01
wandb: 	lr: 0.001
wandb: 	max_epoch: 150
wandb: 	optimizer: adamW
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Init GBDT2NN
Init GBDT2NN succeed!
Evaluate Result:
Epoch-000     8 Batches, Step      8, Testing Loss: 35.362620, Used Time:   0.0m, Remaining Time:   1.0m
-------------------------------------------------------------------------------
Best Metric: 35.36262013960858
####################################################################################
Evaluate Result:
Epoch-001     8 Batches, Step     16, Testing Loss: 34.301700, Used Time:   0.0m, Remaining Time:   1.0m
-------------------------------------------------------------------------------
Best Metric: 34.30170012493523
####################################################################################
Evaluate Result:
Epoch-002     8 Batches, Step     24, Testing Loss: 10.233359, Used Time:   0.0m, Remaining Time:   0.9m
-------------------------------------------------------------------------------
Best Metric: 10.233359083837392
####################################################################################
Evaluate R

batch loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
createdAt,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test min loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch loss,0.10175
createdAt,1644231092.20977
loss,0.18432
test min loss,0.14361


wandb: Agent Starting Run: 9c92yw8a with config:
wandb: 	batch_size: 64
wandb: 	l2_reg: 3e-05
wandb: 	lr: 0.01
wandb: 	max_epoch: 20
wandb: 	optimizer: adamW
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Init GBDT2NN
Init GBDT2NN succeed!
Evaluate Result:
Epoch-000    62 Batches, Step     62, Testing Loss: 32.526074, Used Time:   0.0m, Remaining Time:   0.2m
-------------------------------------------------------------------------------
Best Metric: 32.52607380613989
####################################################################################
Evaluate Result:
Epoch-001    62 Batches, Step    124, Testing Loss: 30.296954, Used Time:   0.0m, Remaining Time:   0.2m
-------------------------------------------------------------------------------
Best Metric: 30.296954096580038
####################################################################################
Evaluate Result:
Epoch-002    62 Batches, Step    186, Testing Loss:  0.387048, Used Time:   0.0m, Remaining Time:   0.2m
-------------------------------------------------------------------------------
Best Metric: 0.387048248733793
####################################################################################
Evaluate R

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=de072003-a9db-4342-8067-19a4b45feff1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>